## Here we reduce and concatenate the stats

In [1]:
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm

def get_fixed_column_names():
    """
    Generate a fixed list of column names to ensure consistency across all files
    """
    columns = []
    
    # Models (excluding model_5)
    models = [f"model_{i}" for i in range(1, 7) if i != 5]
    
    # Metrics for each model
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Statistics to compute
    stats = ['median', 'max', 'min', 'std', 
             'moment1', 'moment2', 'moment3', 'moment4', 'moment5', 
             'q95', 'q90', 'q85', 'q80', 'q20', 'q15', 'q10', 'q05']
    
    # Generate columns for model metrics
    for model in models:
        for metric in metrics:
            for stat in stats:
                columns.append(f"{model}_{metric}_{stat}")
    
    # Generate columns for KL divergence terms
    model_pairs = [(i, j) for i in range(1, 7) for j in range(i+1, 7) 
                  if i != 5 and j != 5]
    
    for i, j in model_pairs:
        for stat in stats:
            columns.append(f"kl_{i}_vs_{j}_{stat}")
    
    return columns

def get_label_from_json(trial_name, question_number, label_folder_path):
    """
    Get label from corresponding JSON file if label_folder_path is provided
    """
    if not label_folder_path:  # If no path provided
        return None
        
    json_path = os.path.join(
        label_folder_path,
        f"{trial_name}.json"
    )
    
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
            label = data[f"Q{question_number}"]["eval_databases"]
            # Explicitly handle "N/A" to prevent pandas from converting it
            if label == "N/A":
                return "N/A"  # Forces pandas to keep it as string
            return label
    except Exception as e:
        print(f"Error reading JSON for {trial_name}: {e}")
        return None

def extract_trial_and_question(file_path):
    """
    Extract trial name and question number from file path
    Example: path/to/NCT00001959/logprob_matrix_1.csv -> ("NCT00001959", 1)
    """
    try:
        # Get filename and remove extension
        filename = os.path.basename(file_path)
        # Extract question number
        question_num = int(filename.split('_')[-1].split('.')[0])
        # Extract trial name from path
        trial_name = file_path.split('/')[-2]  # Adjust this based on your actual path structure
        return trial_name, question_num
    except Exception as e:
        print(f"Error extracting trial and question from {file_path}: {e}")
        return None, None

def compute_statistics(series):
    """
    Compute various statistical measures for a series
    """
    mean = series.mean()
    
    stats = {
        'median': series.median(),
        'max': series.max(),
        'min': series.min(),
        'std': series.std(),
        
        # Central moments
        'moment1': mean,
        'moment2': ((series - mean) ** 2).mean(),
        'moment3': ((series - mean) ** 3).mean(),
        'moment4': ((series - mean) ** 4).mean(),
        'moment5': ((series - mean) ** 5).mean(),
        
        # Existing quantiles
        'q95': series.quantile(0.95),
        'q90': series.quantile(0.90),
        'q85': series.quantile(0.85),
        'q80': series.quantile(0.80),
        'q20': series.quantile(0.20),
        'q15': series.quantile(0.15),
        'q10': series.quantile(0.10),
        'q05': series.quantile(0.05),
    }    
    return stats

def process_metrics_and_kl(input_folder, kl_folder, output_folder, label_folder_path=None):
    """
    Process original metrics and KL divergence terms with consistent column ordering
    """
    # Get fixed column names
    fixed_columns = get_fixed_column_names()
    
    # Original metrics to analyze
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Get all CSV files from model1 directory
    csv_files = []
    model1_path = os.path.join(input_folder, "model_1")
    for root, dirs, files in os.walk(model1_path):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to process")
    print(f"Output will have {len(fixed_columns) + 1} columns")  # +1 for label
    
    for file_path1, file_name in tqdm(csv_files, desc="Processing files"):
        try:
            # Get trial name and question number
            trial_name, question_num = extract_trial_and_question(file_path1)
            
            if trial_name and question_num and label_folder_path:
                # Get label from JSON only if path is provided
                label = get_label_from_json(trial_name, question_num, label_folder_path)
            else:
                label = None
            
            # Dictionary to store all metrics
            all_stats = {col: np.nan for col in fixed_columns}  # Initialize with NaN
            
            # Add label column
            all_stats['label'] = label
            
            # Process each model (excluding model_5)
            models = [f"model_{i}" for i in range(1, 7) if i != 5]
            
            # Step 1: Process original metrics for each model
            for model in models:
                file_path = os.path.join(input_folder, model, os.path.relpath(file_path1, model1_path))
                if not os.path.exists(file_path):
                    print(f"Skipping {file_name} - no matching file in {model}")
                    continue
                
                df = pd.read_csv(file_path)
                
                # Compute statistics for each metric
                for metric in metrics:
                    if metric in df.columns:
                        stats = compute_statistics(df[metric])
                        for stat_name, value in stats.items():
                            col_name = f"{model}_{metric}_{stat_name}"
                            all_stats[col_name] = value
            
            # Step 2: Process KL divergence terms
            kl_file_path = os.path.join(kl_folder, os.path.relpath(file_path1, model1_path))
            if os.path.exists(kl_file_path):
                kl_df = pd.read_csv(kl_file_path)
                
                # Get all KL columns
                kl_cols = [col for col in kl_df.columns if col.startswith('kl_')]
                
                # Compute statistics for each KL term
                for kl_col in kl_cols:
                    stats = compute_statistics(kl_df[kl_col])
                    for stat_name, value in stats.items():
                        col_name = f"{kl_col}_{stat_name}"
                        all_stats[col_name] = value
            
            # Create output DataFrame with fixed column order plus label
            columns_with_label = fixed_columns + ['label']
            result_df = pd.DataFrame([all_stats])[columns_with_label]
            
            # Create output directory structure
            output_file_path = os.path.join(output_folder, os.path.relpath(file_path1, model1_path))
            os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
            
            # Save results
            result_df.to_csv(output_file_path, index=False)
            
        except Exception as e:
            print(f"\nError processing {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue

def concatenate_output_files(output_folder):
    """
    Concatenate all CSV files in Output_folder and its subfolders into a single DataFrame
    """
    # Get list of all CSV files
    csv_files = []
    for root, dirs, files in os.walk(output_folder):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to concatenate")
    # Sort files alphabetically
    csv_files = sorted(csv_files)
    
    # Read and concatenate all files
    all_dfs = []
    for file_path, file_name in tqdm(csv_files, desc="Reading files"):
        try:
            df = pd.read_csv(file_path)
            
            # Add filename as a column (optional but useful for tracking)
            df['source_file'] = file_name
            
            # Add full path as a column (optional)
            df['file_path'] = os.path.relpath(file_path, output_folder)
            
            all_dfs.append(df)
            
        except Exception as e:
            print(f"\nError reading {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue
    
    # Concatenate all DataFrames
    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        
        # Print some information about the final DataFrame
        print("\nFinal DataFrame info:")
        print(f"Shape: {final_df.shape}")
        print(f"Number of samples: {len(final_df)}")
        print(f"Number of features: {len(final_df.columns)}")
        
        if 'label' in final_df.columns:
            print("\nLabel distribution:")
            print(final_df['label'].value_counts())
        
        return final_df
    else:
        print("No files were successfully read!")
        return None

if __name__ == "__main__":
    # Paths
    input_folder = "Database_free_evaluation/Clinical_trials/Features/FEATURES_DeepSeek"
    kl_folder = "kl_analysis_Paragraph_title_features"
    output_folder = "Output_folder_Paragraph_title"

     # Label folder path (set to None if not using labels)
    label_folder_path = "Database_dependent_evaluation/Clinical_trials/4-Evaluation/Evaluation_factual/DeepSeek/Paragraph_level/Evaluation_title_Paragraph_GPT"
    # Or for other folders:
    # label_folder_path = None
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Step 1: Process files and create individual outputs
    process_metrics_and_kl(input_folder, kl_folder, output_folder, label_folder_path)

    # Step 2: Concatenate all output files
    final_df = concatenate_output_files(output_folder)
    
    # Optional: Save concatenated DataFrame
    if final_df is not None:
        final_df.to_csv("concatenated_results_Paragraph_title.csv", index=False)

Found 1499 CSV files to process
Output will have 681 columns


Processing files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1499/1499 [02:41<00:00,  9.29it/s]


Found 1499 CSV files to concatenate


Reading files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1499/1499 [00:10<00:00, 140.45it/s]



Final DataFrame info:
Shape: (1499, 683)
Number of samples: 1499
Number of features: 683

Label distribution:
label
hallucination    1150
fact              347
Name: count, dtype: int64


In [2]:
final_df

,model_1_gold_lp_median,model_1_gold_lp_max,model_1_gold_lp_min,model_1_gold_lp_std,model_1_gold_lp_moment1,model_1_gold_lp_moment2,model_1_gold_lp_moment3,model_1_gold_lp_moment4,model_1_gold_lp_moment5,model_1_gold_lp_q95,...,kl_4_vs_6_q90,kl_4_vs_6_q85,kl_4_vs_6_q80,kl_4_vs_6_q20,kl_4_vs_6_q15,kl_4_vs_6_q10,kl_4_vs_6_q05,label,source_file,file_path
0,-0.000016,0.0,-14.116863,1.650339,-0.376421,2.695541,-29.291987,3.788929e+02,-5.101671e+03,0.000000e+00,...,0.305703,0.202104,0.088715,3.320311e-07,1.460378e-07,4.908980e-08,8.288604e-09,hallucination,logprob_matrix_1.csv,NCT00001959/logprob_matrix_1.csv
1,-0.007329,0.0,-100.000000,10.887073,-2.401454,117.225849,-10379.188407,1.001059e+06,-9.740825e+07,0.000000e+00,...,0.378235,0.266549,0.217734,3.457059e-06,2.413533e-06,4.376253e-07,7.058479e-08,hallucination,logprob_matrix_10.csv,NCT00001959/logprob_matrix_10.csv
2,-0.030774,0.0,-100.000000,10.305379,-2.077365,105.105992,-9703.605439,9.482396e+05,-9.282447e+07,0.000000e+00,...,0.379517,0.281111,0.199646,6.378914e-06,2.484864e-06,4.853980e-07,2.474919e-08,hallucination,logprob_matrix_11.csv,NCT00001959/logprob_matrix_11.csv
3,-0.559920,0.0,-100.000000,20.410269,-6.222168,412.924882,-36061.273966,3.393322e+06,-3.181027e+08,-3.516673e-07,...,0.477269,0.383312,0.314148,6.137047e-03,9.056092e-04,1.569917e-04,1.054056e-05,hallucination,logprob_matrix_12.csv,NCT00001959/logprob_matrix_12.csv
4,-0.005173,0.0,-100.000000,18.055005,-4.175513,320.550147,-29277.356694,2.810752e+06,-2.693146e+08,0.000000e+00,...,0.387154,0.266365,0.183391,2.153387e-06,1.439171e-06,1.348659e-07,2.859716e-08,hallucination,logprob_matrix_13.csv,NCT00001959/logprob_matrix_13.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,-0.000003,0.0,-12.882793,1.950844,-0.699867,3.777389,-27.615960,2.662460e+02,-2.770496e+03,0.000000e+00,...,0.352078,0.236819,0.155296,7.688756e-07,3.125340e-07,5.273651e-08,3.380948e-08,hallucination,logprob_matrix_5.csv,NCT04739800/logprob_matrix_5.csv
1495,-0.018668,0.0,-100.000000,8.843934,-1.545903,77.618114,-7295.455760,7.173414e+05,-7.061592e+07,0.000000e+00,...,0.299087,0.228928,0.146895,3.523717e-06,1.225588e-06,4.676687e-07,2.930865e-08,hallucination,logprob_matrix_6.csv,NCT04739800/logprob_matrix_6.csv
1496,-0.010285,0.0,-10.905843,3.115767,-1.104159,8.899002,-77.539064,7.701772e+02,-7.538058e+03,0.000000e+00,...,0.702936,0.499897,0.357136,4.996134e-07,2.936043e-07,1.522245e-07,7.603017e-08,fact,logprob_matrix_7.csv,NCT04739800/logprob_matrix_7.csv
1497,-0.078338,0.0,-100.000000,12.523537,-2.878957,156.017839,-14388.508151,1.397616e+06,-1.357242e+08,0.000000e+00,...,0.473413,0.392672,0.246967,1.552832e-05,4.076553e-06,1.474322e-06,1.616534e-07,hallucination,logprob_matrix_8.csv,NCT04739800/logprob_matrix_8.csv
